In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import os, pathlib, shutil, random

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim 
        self.dense_dim = dense_dim 
        self.num_heads = num_heads 
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),layers.Dense(embed_dim),])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None): 
        if mask is not None: 
            mask = mask[:, tf.newaxis, :] 
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self): 
        config = super().get_config()
        config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs): 
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding( 
        input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
        input_dim=sequence_length, output_dim=output_dim) 
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None): 
        return tf.math.not_equal(inputs, 0) 
    
    def get_config(self): 
        config = super().get_config()
        config.update({
        "output_dim": self.output_dim,
        "sequence_length": self.sequence_length,
        "input_dim": self.input_dim,
        })
        return config

In [11]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

In [ ]:
#Delete train/unsup directory

In [12]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files) 
    num_val_samples = int(0.2 * len(files)) 
    val_files = files[-num_val_samples:] 
    for fname in val_files: 
        shutil.move(train_dir / category / fname, val_dir / category / fname)

In [6]:
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory( "aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

text_only_train_ds = train_ds.map(lambda x, y: x) 

max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
 max_tokens=max_tokens,
 output_mode="int",
 output_sequence_length=max_length, 
)
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [14]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs) 
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
 loss="binary_crossentropy",
 metrics=["accuracy"])

In [15]:
callbacks = [
 keras.callbacks.ModelCheckpoint("model/full_transformer_encoder_2.keras", save_best_only=True),
keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
] 
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, 
callbacks=callbacks)
model.save("best/full_transformer_encoder_3.keras")
model.save_weights("best/best_weights")

Epoch 1/20
625/625 [==============================] - 57s 90ms/step - loss: 0.4638 - accuracy: 0.7803 - val_loss: 0.3017 - val_accuracy: 0.8792
Epoch 2/20
625/625 [==============================] - 56s 89ms/step - loss: 0.1663 - accuracy: 0.9358 - val_loss: 0.3357 - val_accuracy: 0.8796
Epoch 3/20
625/625 [==============================] - 56s 89ms/step - loss: 0.0923 - accuracy: 0.9659 - val_loss: 0.4202 - val_accuracy: 0.8750
Epoch 4/20
625/625 [==============================] - 68s 109ms/step - loss: 0.0599 - accuracy: 0.9771 - val_loss: 0.5798 - val_accuracy: 0.8646
Epoch 5/20
625/625 [==============================] - 57s 91ms/step - loss: 0.0422 - accuracy: 0.9861 - val_loss: 0.6655 - val_accuracy: 0.8668
Epoch 6/20
625/625 [==============================] - 56s 89ms/step - loss: 0.0329 - accuracy: 0.9886 - val_loss: 0.5906 - val_accuracy: 0.8706


In [16]:
model = keras.models.load_model(
    "model/full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 30s 38ms/step - loss: 0.2898 - accuracy: 0.8779
Test acc: 0.878
